In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [16]:
ALTERNATES = pd.read_pickle('data/temp01/ALTERNATES.pkl')
PMA_ACCEPTANCE = pd.read_pickle('data/temp01/PMA_ACCEPTANCE.pkl')
PARTS_MASTER = pd.read_pickle('data/temp02/PARTS_MASTER.pkl')

PARTS_MASTER.drop_duplicates(subset=['PN'],keep='first',inplace = True)

#bring real PNM
ALTERNATES = ALTERNATES.merge(PARTS_MASTER[['PN','BEST_PNM','CODE']],left_on = 'PN',right_on='PN',how='left')
ALTERNATES.drop(columns=['PNM'],inplace = True)
ALTERNATES = ALTERNATES.merge(PARTS_MASTER[['PN','BEST_PNM','CODE']],left_on = 'ALT_PN',right_on='PN',how='left')
ALTERNATES.drop(columns=['ALT_PNM'],inplace = True)
ALTERNATES.rename(columns={'PN_x':'PN','PNM_x':'PNM','BEST_PNM':'ALT_PNM','CODE_x':'CODE','CODE_y':'ALT_CODE'},inplace = True)
ALTERNATES.drop(columns=['PN_y'],inplace = True)

#Bring back PN score to sort alt by best PN
ALTERNATES = ALTERNATES.merge(PARTS_MASTER[['BEST_PNM','TOTAL_SCORE']],left_on = 'ALT_PNM',right_on='BEST_PNM',how='left')
ALTERNATES.rename(columns={'TOTAL_SCORE':'ALT_SCORE'},inplace = True)
ALTERNATES.drop(columns=['BEST_PNM'],inplace = True)
ALTERNATES.sort_values(by=['PNM','ALT_SCORE'],ascending=False,inplace =True)

#Add PNM to PMA acceptance list
PMA_ACCEPTANCE = PMA_ACCEPTANCE.merge(PARTS_MASTER[['PN','PNM']],left_on='PN',right_on='PN',how='left')
PMA_ACCEPTANCE.rename(columns={'PN':'LRU_PN','PNM':'LRU_PNM','pma':'PN'},inplace=True)
PMA_ACCEPTANCE = PMA_ACCEPTANCE.merge(PARTS_MASTER[['PN','PNM']],left_on='PN',right_on='PN',how='left')

#remove missing PN from master
#TODO : show the dropped lines to the user
PMA_ACCEPTANCE = PMA_ACCEPTANCE[~PMA_ACCEPTANCE['LRU_PNM'].isnull()]
PMA_ACCEPTANCE = PMA_ACCEPTANCE[~PMA_ACCEPTANCE['PNM'].isnull()]
PMA_ACCEPTANCE.drop_duplicates(inplace=True)


PMA_ACCEPTANCE.to_pickle('data/temp02/PMA_ACCEPTANCE.pkl')
ALTERNATES.to_pickle('data/temp02/ALTERNATES.pkl')